In [408]:
import io
import cv2
import numpy as np
import pickle
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import axes3d, Axes3D

In [410]:
real = np.load('/root/PlasticineLab/real2sim/real_points/cloth/black/real_pcds.npy', allow_pickle=True)
with open('/root/GenORM/policy/pbm/experts/Table_1500_1500_0.36_0.36_200_200/Table-v1/expert_0.0025_09:31:14.pickle', 'rb') as f:
    sim = pickle.load(f)
    sim = sim['plasticine_pc']

In [411]:
def rotate_point_cloud_around_center(point_cloud, angle_degrees):
    # Calculate the centroid of the point cloud
    centroid = np.mean(point_cloud, axis=0)

    # Translate the point cloud to the origin
    translated_cloud = point_cloud - centroid

    # Rotation matrix for rotation around y-axis
    cos_angle = np.cos(np.radians(angle_degrees))
    sin_angle = np.sin(np.radians(angle_degrees))
    rotation_matrix = np.array([[cos_angle,  0, sin_angle],
                                [0,         1, 0],
                                [-sin_angle, 0, cos_angle]])

    # Rotate the translated point cloud
    rotated_translated_cloud = np.dot(translated_cloud, rotation_matrix.T)

    # Translate back to the original position
    rotated_cloud = rotated_translated_cloud + centroid

    return rotated_cloud

for i in range(real.shape[0]):
    real[i] = real[i]*2
    
    rotation_matrix = np.array([[1,  0,  0],
                                [0,  0, 1],
                                [0,  1,  0]])
    real[i] = np.dot(real[i], rotation_matrix.T)
    
    mean_offset = sim[i].mean(axis=0) - real[i].mean(axis=0)
    real[i] += mean_offset
    
    real[i] = rotate_point_cloud_around_center(real[i], -140)
    
    if real[i].shape[0] > 5000:
        real[i] = real[i-1]

real = [arr[~np.any(arr > 0.97, axis=1)] for arr in real]
# real = [arr[~np.any(arr < 0.01, axis=1)] for arr in real]
np.save('/root/PlasticineLab/real2sim/real_points/cloth/black/modified_real_pcds.npy', real)

/usr/local/lib/python3.8/dist-packages/numpy/lib/npyio.py:501: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  arr = np.asanyarray(arr)


In [ ]:
# real[6][:, [0, 1, 2]] = real[6][:, [0, 2, 1]]
# real[26][:, [0, 1, 2]] = real[26][:, [0, 2, 1]]

plt.rcParams['font.size'] = 7
vis_index = 150
count = 0
for real_pcds, sim_pcds in zip(real[:vis_index], sim[:vis_index]):
    fig = plt.figure(figsize=(15, 5))
    ax1 = fig.add_subplot(1, 3, 1, projection='3d')
    ax1.view_init(140, -90)
    ax1.set_xlabel('X Label')
    ax1.set_ylabel('Y Label')
    ax1.set_zlabel('Z Label')
    ax1.set_xlim([0.2, 0.8])
    ax1.set_ylim([0, 0.8])
    ax1.set_zlim([0.2, 0.8])
    ax1.set_title(f'{count}th real')
    ax1.scatter(real_pcds[:, 0], real_pcds[:, 1], real_pcds[:, 2], s=1)
    ax1.scatter(sim_pcds[:, 0], sim_pcds[:, 1], sim_pcds[:, 2], s=1)
    
    count += 1

In [ ]:
real_pcds.shape

(638, 3)

In [ ]:
real[0].shape

(637, 3)